# Predicting the Severity of Major Power Outages in the U.S.

**Name(s)**: Pratham Aggarwal

**Website Link**: https://pratham-aggr.github.io/power_outage

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from dsc80_utils import *
import folium
import requests
from scipy import stats
pd.options.plotting.backend = "plotly"
import plotly.io as pio
pio.renderers.default = "iframe"
import plotly.express as px
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.preprocessing import OrdinalEncoder

## Nb Utils

In [3]:
def get_variable_lists():
    categorical_vars = [
        'u.s._state',
        'nerc.region',
        'climate.region',
        'climate.category',
        'cause.category',
        'cause.category.detail'
    ]

    numeric_vars = [
        'anomaly.level (numeric)',
        'demand.loss.mw (megawatt)',
        'customers.affected',
        'res.price (cents / kilowatt-hour)',
        'com.price (cents / kilowatt-hour)',
        'ind.price (cents / kilowatt-hour)',
        'total.price (cents / kilowatt-hour)',
        'res.sales (megawatt-hour)',
        'com.sales (megawatt-hour)',
        'ind.sales (megawatt-hour)',
        'total.sales (megawatt-hour)',
        'res.percen (%)',
        'com.percen (%)',
        'ind.percen (%)',
        'res.customers',
        'com.customers',
        'ind.customers',
        'total.customers',
        'res.cust.pct (%)',
        'com.cust.pct (%)',
        'ind.cust.pct (%)',
        'pc.realgsp.state (usd)',
        'pc.realgsp.usa (usd)',
        'pc.realgsp.rel (fraction)',
        'pc.realgsp.change (%)',
        'util.realgsp (usd)',
        'total.realgsp (usd)',
        'util.contri (%)',
        'pi.util.ofusa (%)',
        'population',
        'poppct_urban (%)',
        'poppct_uc (%)',
        'popden_urban (persons per square mile)',
        'popden_uc (persons per square mile)',
        'popden_rural (persons per square mile)',
        'areapct_urban (%)',
        'areapct_uc (%)',
        'pct_land (%)',
        'pct_water_tot (%)',
        'pct_water_inland (%)'
    ]

    datetime_vars = [
        'outage_start',
        'outage_restore'
    ]

    return categorical_vars, numeric_vars, datetime_vars

def plot_single_bar(df,col,color = 'blue'):
    vc = df[col].value_counts(normalize=True).reset_index()
    vc.columns = [col, 'proportion']
    fig = make_subplots(rows = 1, cols = 1, subplot_titles = [col])
    fig.add_trace(go.Bar(x=vc[col], y=vc['proportion'], marker_color = color), row=1, col=1)
    
    fig.update_layout(
        height=500, 
        width=500,
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)'
    )
    fig.write_html(f'assets/univariate_analysis_{col}.html')
    return fig
    #plotly subplots reference: https://plotly.com/python/subplots

def plot_multiple_bars(df, columns ,title = 'Distributions'):
    n = len(columns)
    cols = 3
    rows = (n + cols - 1) // cols

    fig = make_subplots(rows=rows, cols=cols, subplot_titles=columns)
    row, col = 1, 1
    
    for var in columns:
        vc = df[var].value_counts(normalize=True).reset_index()
        vc.columns = [var, 'proportion']
        fig.add_trace(go.Bar(x=vc[var], y=vc['proportion']), row=row, col=col)
        col += 1
        if col > cols:
            col = 1
            row += 1
            
    fig.update_layout(
        height=500 * rows, 
        width=1200, 
        title=title,
        paper_bgcolor='rgb(243, 243, 243)',
        plot_bgcolor='rgb(243, 243, 243)'
    )
    return fig
    
def plot_state_choropleth(df, value_col, aggfunc = 'mean'):
    map_df = df.groupby('u.s._state')[value_col].agg(aggfunc).reset_index()    
    geojson_url = "https://raw.githubusercontent.com/python-visualization/folium-example-data/main/us_states.json"
    us_states = requests.get(geojson_url).json()
    
    fig = px.choropleth(
        map_df,
        geojson = us_states,
        locations = 'u.s._state',
        featureidkey="properties.name", 
        color = value_col,
        color_continuous_scale = 'YlGn',
        scope = 'usa',
        labels = 'value_col.title()',
        title=f"State-Level {value_col} ({aggfunc})"
        )
    
    fig.update_layout(
        geo=dict(
            lakecolor="rgb(255, 255, 255)",
            bgcolor="rgba(0,0,0,0)"
        ),
        title_x=0.5
    )

    fig.write_html('assets/map.html')
    return fig
    #px choropleth reference: https://plotly.com/python/choropleth-maps


## Step 1: Introduction

In [4]:
# data_dct = https://www.sciencedirect.com/science/article/pii/S2352340918307182
fp = Path('data') / 'outage.csv'
raw_df = pd.read_csv(fp, skiprows=5)

In [5]:
raw_df.shape
#summary stats, col description are in section Step 2: Data Cleaning and Exploratory Data Analysis 
#because the data requires a little bit of cleaning before displaying anything

(1535, 57)

## Step 2: Data Cleaning and Exploratory Data Analysis

In [6]:
df = raw_df.copy(deep=True)
df.columns = [col.lower() for col in df.columns]
units = df.iloc[0]

df = df.iloc[1:].reset_index(drop=True)

new_columns = []
for col, unit in zip(df.columns, units):
    if pd.notna(unit):
        new_columns.append(f"{col} ({unit})")
    else:
        new_columns.append(col)

df.columns = new_columns
df.columns = df.columns.str.lower()

categorical_vars, numeric_vars, datetime_vars = get_variable_lists()

for col in numeric_vars:
    df[col] = df[col].astype(float)
    
#state abbreiation are df['postal.code'] hence drop it
#dropping outage start and restore and making a single col for start time/date and end; month/year is redundant info
#hurrican name is useless for our analysis
df = df.rename(columns={
    "outage.start.date (day of the week, month day, year)": "start_date",
    "outage.start.time (hour:minute:second (am / pm))": "start_time",
    "outage.restoration.date (day of the week, month day, year)": "restore_date",
    "outage.restoration.time (hour:minute:second (am / pm))": "restore_time"
})
fmt = "%A, %B %d, %Y %I:%M:%S %p"

df["outage_start"] = pd.to_datetime(df["start_date"] + " " + df["start_time"], format=fmt)
df["outage_restore"] = pd.to_datetime(df["restore_date"] + " " + df["restore_time"], format=fmt)
df["dur_hours"] = (df["outage_restore"] - df["outage_start"]).dt.total_seconds() / 3600

#since outage.duration is linearly dependent on outage_start & outage_restore, to I will drop it to avoid redundant info
#cause.category.detail is missing by design so even filling the values won't influence things a lot so would drop it
df = df.drop(
    columns=[
        "start_date", 
        "start_time", 
        "restore_date", 
        "restore_time", 
        "year", 
        "month", 
        "hurricane.names",
        "outage.duration (mins)",
        'obs',
        'variables (units)', 
        'postal.code',
        'outage_start',
        'outage_restore',
    ]
)

In [7]:
important_cols = [
    'dur_hours',
    'customers.affected',
    'demand.loss.mw (megawatt)',
    'population',
    'poppct_urban (%)',
    'res.price (cents / kilowatt-hour)',
    'pc.realgsp.state (usd)'
]
df.describe()[important_cols]

,dur_hours,customers.affected,demand.loss.mw (megawatt),population,poppct_urban (%),res.price (cents / kilowatt-hour),pc.realgsp.state (usd)
count,1476.00,1.09e+03,829.00,1.53e+03,1534.00,1512.00,1534.00
mean,43.75,1.43e+05,536.29,1.32e+07,80.97,11.97,49390.12
std,99.07,2.87e+05,2196.45,1.16e+07,11.90,3.09,11687.43
...,...,...,...,...,...,...,...
50%,11.68,7.01e+04,168.00,8.77e+06,84.05,11.46,48370.00
75%,48.00,1.50e+05,400.00,1.94e+07,89.81,13.90,53622.00
max,1811.88,3.24e+06,41788.00,3.93e+07,100.00,34.58,168377.00


In [8]:
plot_multiple_bars(df, categorical_vars ,title = 'Distributions')

In [9]:
plot_single_bar(df,'cause.category')

In [10]:
plot_single_bar(df,'climate.region', 'red')

In [11]:
fig = px.box(
    df,
    x = 'cause.category',
    y = 'dur_hours',
    title = 'Avergae Outage Duration by Casue Category'
)

fig.update_layout(
    height = 500,
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)'
)
fig.update_yaxes(type='log') #for visibility since vanilla plot is not legible
fig.write_html('assets/bivariate_analysis_cause.category_vs_dur_hours_box_plot.html')
fig.show()
#courtesy plotly boxplots reference: https://plotly.com/python/box-plots

In [12]:
df['u.s._state'].unique()

array(['Minnesota', 'Tennessee', 'Wisconsin', ..., 'North Dakota',
       'South Dakota', 'Alaska'], dtype=object)

In [13]:
plot_state_choropleth(df, 'dur_hours', aggfunc = 'mean')

## Step 3: Assessment of Missingness

In [14]:
figs = ['pct_water_tot (%)', 'com.sales (megawatt-hour)']

for col in figs:
    fig = create_kde_plotly(
        df = df, 
        group_col = 'anomaly.level (numeric)', 
        group1 = True, 
        group2 = False, 
        vals_col = col, 
        title=f'KDE: anomaly.level vs {col}'
        )
    fig.show()


def perm_test(df, col1, col2):
    missing = df[df[col1].isna()][col2].values
    not_missing = df[df[col1].notna()][col2].values
    obs = stats.ks_2samp(missing, not_missing).statistic
    comb = np.concatenate([missing, not_missing])
    perm_stats = []
    for _ in range(10_000):
        perm = np.random.permutation(comb)
        perm_miss = perm[:len(missing)]
        perm_not_miss = perm[len(missing):]
        perm_stat = stats.ks_2samp(perm_miss, perm_not_miss).statistic
        perm_stats.append(perm_stat)
    
    perm_stats = np.array(perm_stats)
    p_val = np.mean(perm_stats >= obs)
    return p_val        

pval_no = perm_test(df, 'anomaly.level (numeric)', 'pct_water_tot (%)')
pval_yes = perm_test(df, 'anomaly.level (numeric)', 'com.sales (megawatt-hour)')

print(pval_no, pval_yes)

0.8255 0.0


## Step 4: Hypothesis Testing

In [15]:
#perm test to examine whether outage duration depends on climate.category (there is an obvious yes atm)
df_valid = df.dropna(subset=['climate.category', 'dur_hours']).copy()
df_valid['is_normal_climate'] = (df_valid['climate.category']=='normal')

fig = create_kde_plotly(
    df = df_valid, 
    group_col = 'is_normal_climate', 
    group1 = True, 
    group2 = False, 
    vals_col = 'dur_hours', 
    title=f'KDE: climate.category vs dur_hours'
)
fig.show()

#since dist shape is quite similar, I will be using ks stat
def ks_perm_test_gen(df, col1, group, col2):
    g1 = df[df[col1] == group][col2].dropna().values
    g2 = df[df[col1]!=group][col2].dropna().values
    obs = stats.ks_2samp(g1, g2).statistic
    comb = np.concatenate([g1, g2])
    perm_stats = []
    for _ in range(10_000):
        perm =np.random.permutation(comb)
        perm_g1 = perm[:len(g1)]
        perm_g2 = perm[len(g1):]
        perm_stat = stats.ks_2samp(perm_g1, perm_g2).statistic
        perm_stats.append(perm_stat)
    
    perm_stats = np.array(perm_stats)
    p_val = np.mean(perm_stats >= obs)
    return p_val

pval = ks_perm_test_gen(df_valid, 'is_normal_climate', True, 'dur_hours')
print(pval)

0.0614


## Step 5: Framing a Prediction Problem

In [16]:
# TODO
#predicting dur.hours based on almoast all the feature present in the df

## Step 6: Baseline Model

In [17]:
df = df.drop(columns = ['cause.category.detail'])

In [18]:
categorical_features = [
    'u.s._state',
    'nerc.region',
    'climate.region',
    'climate.category',
    'cause.category',
]

numeric_features = [
    'anomaly.level (numeric)',
    'demand.loss.mw (megawatt)',
    'customers.affected',
    'res.price (cents / kilowatt-hour)',
    'com.price (cents / kilowatt-hour)',
    'ind.price (cents / kilowatt-hour)',
    'total.price (cents / kilowatt-hour)',
    'res.sales (megawatt-hour)',
    'com.sales (megawatt-hour)',
    'ind.sales (megawatt-hour)',
    'total.sales (megawatt-hour)',
    'res.percen (%)',
    'com.percen (%)',
    'ind.percen (%)',
    'res.customers',
    'com.customers',
    'ind.customers',
    'total.customers',
    'res.cust.pct (%)',
    'com.cust.pct (%)',
    'ind.cust.pct (%)',
    'pc.realgsp.state (usd)',
    'pc.realgsp.usa (usd)',
    'pc.realgsp.rel (fraction)',
    'pc.realgsp.change (%)',
    'util.realgsp (usd)',
    'total.realgsp (usd)',
    'util.contri (%)',
    'pi.util.ofusa (%)',
    'population',
    'poppct_urban (%)',
    'poppct_uc (%)',
    'popden_urban (persons per square mile)',
    'popden_uc (persons per square mile)',
    'popden_rural (persons per square mile)',
    'areapct_urban (%)',
    'areapct_uc (%)',
    'pct_land (%)',
    'pct_water_tot (%)',
    'pct_water_inland (%)'
]

target = 'dur_hours'
df = df.dropna(subset=['dur_hours'])
X = df.drop(columns = ['dur_hours'])
y = df['dur_hours']

In [ ]:
# TODO
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV, train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

cat_proc = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'constant', fill_value = 'Unknown')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

num_proc = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy = 'median')),
    ('scale', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_proc, numeric_features),
        ('cat', cat_proc, categorical_features)
    ]
)

model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LGBMRegressor(random_state=42))
])

try_params = {
    'model__n_estimators': [300, 600, 900],
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__num_leaves': [31, 63, 127],
    'model__max_depth': [-1, 10, 20, 40],
    'model__min_child_samples': [5, 10, 20],
    'model__subsample': [0.7, 1.0],
    'model__colsample_bytree': [0.7, 1.0],
}
grid = GridSearchCV(
    estimator=model,
    param_grid=try_params,
    cv=3,
    n_jobs=-1,
    verbose=2
)

grid.fit(X_train, y_train)

Fitting 3 folds for each of 1296 candidates, totalling 3888 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.014168 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6488
[LightGBM] [Info] Number of data points in the train set: 786, number of used features: 102
[LightGBM] [Info] Start training from score 41.413974
[CV] END model__colsample_bytree=0.7, model__learning_rate=0.01, model__max_depth=-1, model__min_child_samples=5, model__n_estimators=300, model__num_leaves=31, model__subsample=0.7; total time=  10.5s
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000630 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6572
[LightGBM] [Info] Number of data points in the train set: 787, number of used features: 105
[LightGBM] [Info] Start training from score 43.784943
[LightGBM] [Warning] No further splits with posit

In [ ]:
grid.best_params_

In [ ]:
grid.predict(X_test)

In [ ]:
grid.score(X_test, y_test)

## Step 7: Final Model

In [ ]:
# TODO
grid.score(X_test, y_test)

## Step 8: Fairness Analysis

In [ ]:
# TODO